# Project EDADIL

## development notebook

This is a training pet project to practice Data Engeneering skills and some personal use. The idea is to get info from edadeal.ru (by parsing it) on daily bases to decide if there are any commodities on sale worth bying...

### ETL schema

Extract - from edadeal.ru - by parsing

Transform - make it as much unified as possible to load it to the DB (probably) PostgreSQL

Load - to PostgreSQL DB.

### Other plans
telegram bot

to store images

#### Import

In [1]:
from datetime import datetime
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import bs4 as bs
import pandas as pd

#### Variables

In [2]:
fp_url = "https://edadeal.ru/podolsk/offers?page=1"
driver_path = "D:\\miniconda\\edadilproject\\Needed items\\chromedriver"
html_path = "D:\\miniconda\\edadilproject\\Data\\raw_html_"
log_path = "D:\\miniconda\\edadilproject\\Data\\log_"
table_path = "D:\\miniconda\\edadilproject\\Data\\table_"

page_iterator = 1
columns_descr = ["badge", "dates", "retailer", "description", "quantity", "price_old", "price_new"]

#### Extract functions

In [3]:
def get_timestamp():
    timestamp = datetime.now()
    timestamp = timestamp.strftime("%Y.%m.%d %H-%M-%S")
    return timestamp

In [4]:
def go_to_page(driver, url):
    if url != None:
        driver.get(url)
    else:
        try:
            next_button = driver.find_element(By.LINK_TEXT, "Вперед →")
        except NoSuchElementException as err:
            return False
        else:
            next_button.click()
            
    # alcohol visibility
            
    driver.implicitly_wait(5)
    time.sleep(5)
    return True

In [5]:
def get_html(driver):
    body = driver.find_element(By.TAG_NAME, "body")
    html = body.get_attribute("innerHTML")
    return html

In [6]:
def save_html(html, path):
    with open(path, "a", encoding="utf-8") as file:
        file.write(html + "\n" + "\n")

In [7]:
def log(text, path):
    log_line = "\n" + get_timestamp() + " " + text
    with open(path, "a") as file:
        file.write(log_line)

In [8]:
def extract(driver, url, html_path, log_path, page_iterator):
    log("Extracting page: " + str(page_iterator), log_path)
    if page_iterator != 1:
        url = None
    result = go_to_page(driver, url)
    if result:
        html = get_html(driver)
        save_html(html, html_path)
        log("Extracting finished", log_path)
        return html, True
    else:
        log("Previous page was the last one.", log_path)
        return None, False


#### Transform functions

In [25]:
def get_contents(item, class_name):
    elements_list = item.select("." + class_name)
    if len(elements_list) > 0:
        text_list = elements_list[0].contents
        if len(text_list) > 0:
            text = text_list[0]
        return text

In [10]:
def read_offer(offer):
    badge = get_contents(offer, "b-offer__badge")
    dates = get_contents(offer, "b-offer__dates")
    class_search_text = "b-image b-image_disabled_false b-image_cap_f b-image_img_vert b-image_loaded_true b-offer__retailer-icon"
    retailer = offer.find_all("div", class_ = class_search_text)[0]['title']
    description = get_contents(offer, "b-offer__description")
    quantity_tag = offer.find_all("div", class_ = "b-offer__quantity")
    if len(quantity_tag) > 0:
        quantity = quantity_tag[0].contents[1]
    else:
        quantity = ""
    price_old = get_contents(offer, "b-offer__price-new")
    price_new = get_contents(offer, "b-offer__price-old")
    data_dict = {"badge":[badge], "dates":[dates], "retailer":[retailer], "description":[description], \
                 "quantity":[quantity], "price_old":[price_old], "price_new":[price_new]}
    df_offer = pd.DataFrame.from_dict(data_dict)
    return df_offer

In [11]:
def transform(html, table_path, df):
    soup = bs.BeautifulSoup(html, "html.parser")
    offer_list = soup.find_all("a", "p-offers__offer")
    for offer in offer_list:
        df_offer = read_offer(offer)
        pd.concat([df, df_offer], ignore_index=True)
    return df 

#### Preparation


In [12]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
service = ChromeService(executable_path=driver_path)
driver = webdriver.Chrome(service=service, options=options)

timestamp = get_timestamp()
log_path = log_path + timestamp + ".txt"
html_path = html_path + timestamp + ".txt"
table_path = table_path + timestamp + ".xlsx"

df = pd.DataFrame(columns=columns_descr)

#### ETL

In [13]:
log("ETL process started", log_path)
result = True
while result and page_iterator < 3:
    html, result = extract(driver, fp_url, html_path, log_path, page_iterator)
   #transform(html, table_path)
    page_iterator += 1
log("ETL process finished", log_path)
driver.close()

In [88]:
#def transform(html, table_path):
df = pd.DataFrame(columns=columns_descr)
soup = bs.BeautifulSoup(html, "html.parser")
offer_list = soup.find_all("a", "p-offers__offer")
for offer in offer_list:
    df_offer = read_offer(offer)
    df = pd.concat([df, df_offer], ignore_index=True)
print(df.head())

    badge          dates          retailer  \
0    -10%  По 20 апреля   Eurospar express   
1  Онлайн  По 20 апреля            М.Видео   
2    -44%  По 30 апреля           ВкусВилл   
3    -23%  По 23 апреля        Перекрёсток   
4  Онлайн    По 16 июля           Любимчик   

                                         description                 quantity  \
0                                Ананас Медовый 1 Кг    1 кг / 299,90 ₽ за кг   
1             БЗУ Rombica NEO Deskstation (NQ-00910)                            
2   Сорбет Баскин Роббинс Клюквенно-малиновый 500 мл   500 мл / 678,00 ₽ за л   
3  Смесь КОМПОТ БЕЗ ХЛОПОТ для приготовления комп...     4 шт / 24,98 ₽ за шт   
4  Perfect Fit влажный корм для котят от 1 до 12 ...    75 г / 400,00 ₽ за кг   

    price_old   price_new  
0    299,90 ₽    329,90 ₽  
1  3 999,00 ₽  4 999,00 ₽  
2    339,00 ₽    599,00 ₽  
3     99,90 ₽    129,00 ₽  
4     30,00 ₽     31,00 ₽  
